# K nearest Neighbour

This is an implementation of K nearest neigbour from scratch

I build a model to address the famous kaggle titanic problem, a binary classification problem. Individuals must be predicted as having survived or not survived the titanic disaster.

In [7]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score
from numpy.random import randint
from scipy import stats

%matplotlib inline
pd.set_option('max.rows', None)

### read data

In [8]:
data = pd.read_csv('train.csv')

In [9]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
feats = data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']]

In [58]:
class_labels = np.array(data['Survived'])

### impute Age NA values with mean  

In [11]:
feats.Age = feats.Age.fillna(data.Age.mean())

In [12]:
feats.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


### defining variables

the features x are an n dimensional matrix:
$$x \in \mathbb{R}^{n}$$
    

    


* scale variables
* choose distance metric
* identify closest K neighbours
* calculate mode class

class attributes will be:
* classification 
* feature 1 value, feature 2 value ... feature j value 

approach 2: vectorised

* store feature data in a numpy array then calculate distance metrics using vectorised operations

my prediction is that approach 2 will be fastest

distance metrics to be implemented:
* euclidean
* manhattan
* Minkowski
* mahalanobis

The algorithm type for calculating the nearest neighbours will be brute force

### distance functions

In [ ]:
def euclidean_distance():
    
def manhattan_distance():
    
def minkowski_distance():
    
def mahalanobis():
    
def test_distance

In [ ]:
def compute_nearest_neighbours(train, test, distance_metric):

In [13]:
feats.head()

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


In [14]:
feats_array = np.array(feats)

In [15]:
feats_array

array([[ 3.        , 22.        ,  1.        ,  0.        ,  7.25      ],
       [ 1.        , 38.        ,  1.        ,  0.        , 71.2833    ],
       [ 3.        , 26.        ,  0.        ,  0.        ,  7.925     ],
       ...,
       [ 3.        , 29.69911765,  1.        ,  2.        , 23.45      ],
       [ 1.        , 26.        ,  0.        ,  0.        , 30.        ],
       [ 3.        , 32.        ,  0.        ,  0.        ,  7.75      ]])

In [21]:
example_unknown = feats_array[0,:]

In [22]:
example_unknown

array([ 3.  , 22.  ,  1.  ,  0.  ,  7.25])

In [119]:
n_neighbours = 5

differences = feats_array-example_unknown
euc_dists = ((differences[:,0])**2+(differences[:,1])**2+(differences[:,2])**2+(differences[:,3])**2+(differences[:,4])**2)**0.5
dist_joined = np.stack((euc_dists,class_labels), axis = -1)
sorter = dist_joined[:,0]
sorted_indexes = sorter.argsort()
sorted_distances = dist_joined[sorted_indexes]
mode_class = stats.mode(sorted_distances[n_neighbours, 1])[0][0]



In [116]:
a = stats.mode([1,2,3,44,44])[0][0]

In [117]:
a

44

In [103]:
sorted_distances[0:10,:]

array([[0.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [1.        , 1.        ],
       [1.0002163 , 0.        ],
       [1.00031245, 1.        ],
       [1.00778222, 0.        ],
       [1.03601768, 0.        ],
       [1.11803399, 1.        ],
       [1.11803399, 1.        ]])

In [96]:
dist_joined

array([[ 0.        ,  0.        ],
       [66.03229141,  1.        ],
       [ 4.17799294,  1.        ],
       ...,
       [18.04761515,  0.        ],
       [23.20694939,  1.        ],
       [10.0623059 ,  0.        ]])

In [87]:
test1 = np.array([[1,2,3,4], [3,2,4,3]]).T

In [88]:
test1

array([[1, 3],
       [2, 2],
       [3, 4],
       [4, 3]])

In [89]:
test1[:,1].shape

(4,)

In [90]:
sorter = test1[:,1]

In [93]:
indexes = sorter.argsort()

In [94]:
indexes

array([1, 0, 3, 2])

In [95]:
test1[indexes]

array([[2, 2],
       [1, 3],
       [4, 3],
       [3, 4]])

# need to include classes in sorted list so can look up what classes